In [1]:
import os
import numpy as np
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.linalg import SparseVector
from scipy.sparse import load_npz


os.environ['PYSPARK_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\Scripts\ipython.exe'

spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc=spark.sparkContext
spark

c:\ProgramData\mambaforge\envs\ML-base\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
df_id=pd.read_parquet("ids_nfcorpus.parquet")
#vocab=pd.read_parquet("terms_nfcorpus.parquet")
threshold=0.8

In [3]:
sparse_repr=load_npz("sparse_repr_nfcorpus.npz")

In [4]:
def csr_to_sparse_vector(row):
    return SparseVector(row.shape[1], list(zip(row.indices, row.data)))

docs = [csr_to_sparse_vector(sparse_repr.getrow(i)) for i in range(sparse_repr.shape[0])]
doc_ids = df_id["_id"]

In [5]:
def sparse_argsort(matrix, idx):
    row=matrix.getrow(idx)
    sorted_indices = np.argsort(row.data)[::-1]
    return row.indices[sorted_indices]
sorted_index_term_doc=[sparse_argsort(sparse_repr, idx) for idx in range(sparse_repr.shape[0])]

In [ ]:
#For debugging
#np.max(sparse_repr.getcol(0))
#np.max(sparse_repr, axis=0).toarray()

In [77]:
#TODO check if is right
d_star=sparse_repr.max(axis=0).tocsr()
b_d=[]
for i in range(sparse_repr.shape[0]):
    term_order=sorted_index_term_doc[i]
    sparse_repr_tmp=sparse_repr[i, term_order]
    d_star_tmp=d_star[0, term_order]
    mult = sparse_repr_tmp.multiply(d_star_tmp).toarray()
    cum_sum=np.cumsum(mult)
    index = np.argmax(cum_sum[cum_sum < threshold])
    b_d.append(term_order[index]) #b(d) value
b_d=np.array(b_d)

In [84]:
b_d

array([5609])

In [ ]:
rdd_forMap=sc.parallelize([(doc_ids[i], (docs[i], sorted_index_term_doc[i], b_d[i])) for i in range(sparse_repr.shape[0])])

In [ ]:
rdd_forMap.first()

Map( (doc_id, (doc, sorted_index_term_doc, b_d)) )
    return (index_term, (doc_id, doc) )

GroupBy: done by Spark

Reduce( (index_term,  list( (doc_id, doc) )) )



In [ ]:
def my_map(elem):
    for t_idx in elem[1][1]:
        if t_idx>elem[1][2]:
            return (t_idx, (elem[0], elem[1][0]))
rdd_forReduce=rdd_forMap.map(my_map)

In [ ]:
tmp=rdd_forReduce.collect()

In [ ]:
tmp[0]

In [ ]:
def array_intersection(list1, list2):
    intersection = []
    i = 0
    j = 0
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            intersection.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1
    return intersection

def my_reduce(elem):
    print(elem)
    for id1, d1 in elem[1]:
        for id2, d2 in elem[1]:
            #TODO: review the comparison with the max (they are index!) check this
            if elem[0]==max(array_intersection(d1.indices, d2.indices)):
                sim=d1.dot(d2)/(d1.norm(2)*d2.norm(2))
                print(sim)
                if sim>=threshold:
                    return (id1, id2, sim)
                
from collections import OrderedDict

def reduce_by_key(ls):
    d = OrderedDict()
    for key, sublist in ls:
        d.setdefault(key, []).extend([sublist])
    return list(d.items())
iter=reduce_by_key(tmp)

In [ ]:
res=map(my_reduce, iter)

In [ ]:
lista=[]
for e in res:
    lista.append(e)

np.unique(lista)

In [ ]:
def my_reduce(elem):
    for id1, d1 in elem[1]:
        for id2, d2 in elem[1]:
            if elem[0]==max(d1.intersect(d2)):
                sim=d1.dot(d2)/(d1.norm()*d2.norm())
                if sim>=threshold:
                    return (id1, id2, sim)
                    

result=rdd_forReduce.reduceByKey(my_reduce)

In [ ]:
result.first()

In [ ]:
spark.stop()